In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pymemri.pod.client import PodClient
from pymemri.data.schema import Account, EmailMessage

In [ ]:
client = PodClient()
client.add_to_schema(EmailMessage)
client2 = PodClient(owner_key=client.owner_key, database_key=client.database_key)

## Default schema

In [ ]:
accounts = [Account(identifier="a") for _ in range(10)]
client.bulk_action(accounts)

%timeit client.create(Account(identifier="a"))
%timeit client.get(accounts[0].id)
%timeit client.search({"type": "EmailMessage", "_limit": 10})

BULK: Writing 10/10 items/edges
Completed Bulk action, written 10 items/edges
4.5 ms ± 61.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.21 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.02 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Central schema

In [ ]:
from pymemri.pod.client import PodClient
from pymemri.data.schema import Account, EmailMessage

client = PodClient()
client.add_to_schema(EmailMessage)

#client.bulk_action(create_items=[EmailMessage(content="a"*1000) for _ in range(100)])
messages = [EmailMessage(content="a"*1000) for _ in range(100)]
for msg in messages:
    client.create(msg)

In [ ]:
%timeit client.create(EmailMessage(content="a"*1000))
%timeit client.get(messages[0].id)
%timeit client.search({"type": "EmailMessage", "_limit": 10})

4.86 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.74 ms ± 55.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.85 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Issue: local_db slows search down significantly
note: only with items added with create (not bulk_action)

In [ ]:
%timeit client.search({"type": "EmailMessage"})
client.reset_local_db()
%timeit client.search({"type": "EmailMessage"})

132 ms ± 3.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
77.9 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
